## 1.1 - Movielens full - EDA

---



 a) Gör en EDA för att förstå datasetet. Inkludera olika slags plots. Begränsa dig inte till frågorna nedan,
utan försök undersöka fler aspekter av datan.
---

Jag startar med att titta över datan så att jag kan visualisera det bättre med planen att skriva ut filmer ihopsatta med deras respektive genre.

In [5]:
import pandas as pd

movies_path = "../../data/ml-latest/movies.csv"

movies_df = pd.read_csv(movies_path)

print(movies_df.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


Med denna utskrift så kan jag se att den innehåller tre stycken olika kolumner "movieId", "title" och "genres", vi kan dock se att genre delas med ett | planen är då att ta ut den relevanta med att splitta från varje | som förekommer för att få fram god data. ChatGPT gav mig ett tips om att använda Counter för att räkna relevant data, hittade tyvärr inte dom relevanta kommandona för det så jag skriver det med hjälp av just GPT men i lärningssyfte att jag ska veta vad jag gör. 

In [13]:
from collections import Counter # ChatGPT gav mig ett gott tips att använda Counter här
import pandas as pd

movies_path = "../../data/ml-latest/movies.csv"
movies_df = pd.read_csv(movies_path)

#Splitta genrerna från | och skapa en lista med alla genre
genre_list = "|".join(movies_df["genres"]).split("|") 

# Räkna varje del av genrerna
genre_counts = Counter(genre_list)

# Konverterar genre_counts till df för visualisering
genre_counts_df = pd.DataFrame(genre_counts.items(), columns = ["Genre", "Movies"])

# Sortera df efter antalet filmerna i descending order
genre_counts_df = genre_counts_df.sort_values(by="Movies", ascending = False).reset_index(drop=True) 

genre_counts_df

,Genre,Movies
0,Drama,33681
1,Comedy,22830
2,Thriller,11675
3,Romance,10172
4,Action,9563
5,Documentary,9283
6,Horror,8570
7,(no genres listed),7060
8,Crime,6917
9,Adventure,5349
